# Mask Tutorial

In this tutorial, we go over the importance of masks, as well as how they are made and used. 

## 0. Importing specpolFlow and other packages

In [ ]:
## Importing Necessary Packages
import specpolFlow as pol
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## 1. What is a Mask?

Analytically, a mask is a function with dirac deltas at wavelengths corresponding to specific spectral line. The amplitude of the dirac delta function corresponds to the line depth. Numerically, a mask is an array of zeros with a depth at the center of each line. Thus, the mask tells us the location and depth of all lines in a spectrum but does not tell us about the shape of the lines of the spectrum as a whole.

## 2. Why do we care?

Given an LSD profile and a mask, we can convolve the LSD profile with the mask to get the spectrum. Typically though, we have the spectrum and a mask but want the LSD profile. This process of going from a spectrum and a line mask to a LSD profile can be done by doing the reverse operation called a deconvolution. We need a mask to help us weigh each spectral line in the spectrum so that they can be averaged together in an LSD.

Note that the hydrogen lines are automatically exluded when `atomsOnly` is `True`. This is done since the hydrogen lines have different shape than all the other lines in the spectrum due to their broad wings.

## 3. Mask Creation

To create a mask we will use the `make_mask` function. The function has the following inputs:
* lineListFile - name of the file containing the line list
* maskFile - name of the file to write the output mask to
* depthCutoff - only include lines in the mask that are deeper than this value
* wlStart - optional, use only lines with wavelengths above this
* wlEnd - optional, use only lines with wavelengths below this
* landeStart - optional, use only lines with effective Lande factors above this
* landeEnd - optional, use only lines with effective Lande factors below this
* elementsUsed - optional, list of elements to include in the mask creation
* elementsExclude - optional, list of elements to exlude from the mask creation
* atomsOnly - only include atomic lines (no molecular lines and no H-lines)
* includeNoLande - include lines in the mask even if they don't have an effective Lande factor given
* DefaultLande - the value for the effective Lande factor to use in the case that a line has no listed effective Lande factor 

For most cases you will only use the top three inputs and `atomsOnly`. The input line list is a VALD line list file and is obtained from the VALD website. More details about VALD is given in `OneObservationFlow_Tutorial.ipynb`. The `make_mask` function will automatically attempt to calculate the effective Lande factor for lines where it is not given, however if one is unable to be calculated it will be excluded if `includeNoLande` is `False` or it will equal `DefaultLande` if `includeNoLande` is `True`.

In the example case used below, we are using all atomic lines in the line list except ones without effective Lande factors and the H-lines. 


In [ ]:
LineList_file_name = 'OneObservationFlow_tutorialfiles/LongList_T27000G35.dat'
Mask_file_name = 'OneObservationFlow_tutorialfiles/test_output/T27000G35_depth0.02.mask'

mask_clean = pol.make_mask(LineList_file_name, Mask_file_name, depthCutoff = 0.02, atomsOnly = True)

## 4. Mask Cleaning

Once we have our mask, we need to clean it. Mask cleaning is the removal of lines that we don't want to be used in the LSD profile. Typically, excluded lines include all lines within the Telluric regions as well as lines within the H wings. Lines within the Telluric regions are contaminated by Earth's atmosphere and thus are not useable. Lines in the H wings are blended with Hydrogen lines thus they are contaminated and unusable since they have a different shape (will effect averaging). Additionally, when dealing with stars with emission, care should be taken to exclude the emission lines if they have different shapes.

In this tutorial, we will use some already defined regions to clean the mask (see the `ExcludeMaskRegionClass_Tutorial` for more details).

In [ ]:
# inputs
velrange = 600 # units are in km/s
excluded_regions = pol.get_Balmer_regions_default(velrange) + pol.get_telluric_regions_default()

# display the excluded regions using Pandas
pd.DataFrame(excluded_regions.to_dict())

Once we have our excluded regions, we can clean the mask using the `mask.clean` function. This function takes in the name of the uncleaned mask, the name of the output mask file, and the dictionary containing the excluded regions. The ouput is a cleaned line mask; lines lying within the `excluded_regions` have been removed.


In [ ]:
# reading in the mask that we created earlier
mask = pol.read_mask('OneObservationFlow_tutorialfiles/test_output/T27000G35_depth0.02.mask')

# applying the ExcludeMaskRegions that we created
mask_clean = mask.clean(excluded_regions)

# saving the new mask to a file
mask_clean.save('OneObservationFlow_tutorialfiles/test_output/hd46328_test_depth0.02_clean.mask')

## 5. Other useful tools

SpecpolFlow also includes an interactive tool to visually inspect a spectrum and select/deselect lines, and compare with the model spectrum from an LSD profile calculated on-the-fly

Additionally, the Mask class has functions to `prune` the mask object (remove from the list all lines that have iuse=0)